<a href="https://colab.research.google.com/github/nirmal-ank/Transformers/blob/main/Introduction_to_GPT_family.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install bertviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 7.2 MB/s eta 0:00:00


In [3]:
from transformers import pipeline, set_seed, GPT2Tokenizer, GPT2LMHeadModel
from torch import tensor, numel #numel for counting parameters
from bertviz import model_view

set_seed(42)

In [21]:
generator = pipeline('text-generation', model='gpt2') #task is text generation

generator("Hello, I'm Donald Trump", max_length=30, num_return_sequences=3)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm Donald Trump. I'm president of the United States of America.\n\nI know that I'm a little bit crazy. I"},
 {'generated_text': 'Hello, I\'m Donald Trump and I\'m my daddy," said Bill Clinton when he took over the White House.\n\nThe billionaire businessman is one'},
 {'generated_text': 'Hello, I\'m Donald Trump and you\'re a liberal? So, you know, you just know, it\'s a race to the bottom."\n'}]

In [23]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
'Biden' in tokenizer.get_vocab()

False

In [24]:
tokenizer.convert_ids_to_tokens(tokenizer.encode('Donald is studying about transformers.'))


['Donald', 'Ġis', 'Ġstudying', 'Ġabout', 'Ġtransform', 'ers', '.']

In [25]:
encoded = tokenizer.encode('Donald is studying about transformers.', return_tensors='pt')

encoded

tensor([[ 7371,   318, 11065,   546,  6121,   364,    13]])

In [26]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [27]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [28]:
model.transformer.wte(encoded).shape #batch size, tokens, length of vector

torch.Size([1, 7, 768])

In [30]:
model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5,6]).reshape(1, 7)).shape

torch.Size([1, 7, 768])

In [32]:
initial_input = model.transformer.wte(encoded) + model.transformer.wpe(tensor([0, 1, 2, 3, 4, 5,6]).reshape(1, 7))
initial_input.shape

torch.Size([1, 7, 768])

In [33]:
initial_input = model.transformer.drop(initial_input)
initial_input

tensor([[[ 0.1169, -0.1829,  0.3222,  ..., -0.2385, -0.0459,  0.1179],
         [ 0.0142, -0.0437, -0.0393,  ...,  0.1487, -0.0278, -0.0255],
         [ 0.0014, -0.0709,  0.0796,  ...,  0.1110,  0.1173, -0.0757],
         ...,
         [ 0.0676, -0.1802,  0.3566,  ...,  0.0323,  0.0369, -0.0256],
         [-0.1741, -0.1992,  0.2401,  ..., -0.0136,  0.0087, -0.0699],
         [ 0.0493, -0.0318,  0.1479,  ..., -0.0710,  0.0533,  0.0938]]],
       grad_fn=<AddBackward0>)

In [34]:
model.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [35]:
for module in model.transformer.h:
    initial_input = module(initial_input)[0] #iterating over all decoder layer

initial_input = model.transformer.ln_f(initial_input)

In [36]:
(initial_input == model(encoded, output_hidden_states=True).hidden_states[-1]).all() #same output through model and layers

tensor(True)

In [37]:
total_params = 0
for param in model.parameters():
    total_params += numel(param)

print(f'Number of params: {total_params:,}')

Number of params: 124,439,808
